In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import nltk
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.orm import Session
import pymysql
pymysql.install_as_MySQLdb()
Base = declarative_base()
from nltk.tokenize import word_tokenize
from sqlalchemy.ext.automap import automap_base
from nltk.stem import PorterStemmer
from textblob import Word
import textblob
from nltk import ngrams
from nltk.util import ngrams

In [2]:
customer_comment=[]
customer_rating=[]
customer_name=[]
review_date=[]

In [3]:
page_count = 0
while page_count < 4:
    
    url = "https://www.consumeraffairs.com/finance/royal_bank.html?page=%s" %(page_count)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    reviews=soup.find(id="reviews-container")
    
    comments=soup.find_all("div", class_="rvw-bd ca-txt-bd-2")
    for ct in comments:
        customer_comment.append(ct.find("p").get_text())
    
    star=reviews.find_all("div",class_="rvw__hdr-stat")
    for st in star:
        customer_rating.append(st.find("img")["data-rating"])

    names=reviews.find_all("div",class_="rvw-aut__inf")
    for n in names:
        customer_name.append(n.find("strong",class_="rvw-aut__inf-nm").get_text())
    
    dates=reviews.find_all("div",class_="rvw-bd ca-txt-bd-2")
    for d in dates:
        review_date.append(d.find("span", class_="ca-txt-cpt ca-txt--clr-gray").get_text())
    
    page_count += 1
    

In [4]:
RBC_Reviews = pd.DataFrame({
"Name": customer_name,
"Rating": customer_rating,
"Date_Reviewed": review_date,
"Review":customer_comment
})


RBC_Reviews.head(20)

,Name,Rating,Date_Reviewed,Review
0,"Tania of London, ON",4.0,"Original review: May 23, 2019",I have found their staff to be extremely helpf...
1,"Terence of Thirnhill, ON",1.0,"Original review: May 23, 2019",Extremely rude people work on the other end of...
2,"John of Fort Erie, ON",2.0,"Original review: May 16, 2019",I've been a RBC client for nearly 20 years and...
3,"Keith of Calgary, AB",1.0,"Original review: May 10, 2019","Wow, I have never dealt with a more unprofessi..."
4,"Salman of Hamilton, ON",1.0,"Original review: May 5, 2019",Business owners stay away from this place. An ...
5,"marco of Hamilton, ON",1.0,"Original review: May 4, 2019",The bank put a deposit only message on my acco...
6,"Sabah of Vancouver, BC",1.0,"Original review: April 30, 2019",I have a business account with RBC. I called t...
7,"Pam of Brampton, ON",1.0,"Original review: April 30, 2019",The franchising come with everything in a neat...
8,"KAREN of White Rock, BC",1.0,"Original review: April 29, 2019",If RBC read any of these reviews they would ha...
9,"Jazz of Other, Other",1.0,"Original review: April 22, 2019",I have business account with RBC. Where I depo...


In [5]:
RBC_Reviews["Date_Reviewed"]=RBC_Reviews["Date_Reviewed"].str.lstrip('Original review:')
RBC_Reviews

,Name,Rating,Date_Reviewed,Review
0,"Tania of London, ON",4.0,"May 23, 2019",I have found their staff to be extremely helpf...
1,"Terence of Thirnhill, ON",1.0,"May 23, 2019",Extremely rude people work on the other end of...
2,"John of Fort Erie, ON",2.0,"May 16, 2019",I've been a RBC client for nearly 20 years and...
3,"Keith of Calgary, AB",1.0,"May 10, 2019","Wow, I have never dealt with a more unprofessi..."
4,"Salman of Hamilton, ON",1.0,"May 5, 2019",Business owners stay away from this place. An ...
5,"marco of Hamilton, ON",1.0,"May 4, 2019",The bank put a deposit only message on my acco...
6,"Sabah of Vancouver, BC",1.0,"April 30, 2019",I have a business account with RBC. I called t...
7,"Pam of Brampton, ON",1.0,"April 30, 2019",The franchising come with everything in a neat...
8,"KAREN of White Rock, BC",1.0,"April 29, 2019",If RBC read any of these reviews they would ha...
9,"Jazz of Other, Other",1.0,"April 22, 2019",I have business account with RBC. Where I depo...


In [6]:
Province=[]
for x in RBC_Reviews['Name']:
    Province.append(x.split(',', 1)[-1])
RBC_Reviews["Province"]=Province
RBC_Reviews

,Name,Rating,Date_Reviewed,Review,Province
0,"Tania of London, ON",4.0,"May 23, 2019",I have found their staff to be extremely helpf...,ON
1,"Terence of Thirnhill, ON",1.0,"May 23, 2019",Extremely rude people work on the other end of...,ON
2,"John of Fort Erie, ON",2.0,"May 16, 2019",I've been a RBC client for nearly 20 years and...,ON
3,"Keith of Calgary, AB",1.0,"May 10, 2019","Wow, I have never dealt with a more unprofessi...",AB
4,"Salman of Hamilton, ON",1.0,"May 5, 2019",Business owners stay away from this place. An ...,ON
5,"marco of Hamilton, ON",1.0,"May 4, 2019",The bank put a deposit only message on my acco...,ON
6,"Sabah of Vancouver, BC",1.0,"April 30, 2019",I have a business account with RBC. I called t...,BC
7,"Pam of Brampton, ON",1.0,"April 30, 2019",The franchising come with everything in a neat...,ON
8,"KAREN of White Rock, BC",1.0,"April 29, 2019",If RBC read any of these reviews they would ha...,BC
9,"Jazz of Other, Other",1.0,"April 22, 2019",I have business account with RBC. Where I depo...,Other


In [7]:
RBC_Reviews['Review'] = RBC_Reviews['Review'].apply(lambda x: " ".join(x.lower() for x in x.split()))
RBC_Reviews['Review'].head()

0    i have found their staff to be extremely helpf...
1    extremely rude people work on the other end of...
2    i've been a rbc client for nearly 20 years and...
3    wow, i have never dealt with a more unprofessi...
4    business owners stay away from this place. an ...
Name: Review, dtype: object

In [8]:
RBC_Reviews['Review'] = RBC_Reviews['Review'].str.replace('[^\w\s]','')
RBC_Reviews['Review'].head()

0    i have found their staff to be extremely helpf...
1    extremely rude people work on the other end of...
2    ive been a rbc client for nearly 20 years and ...
3    wow i have never dealt with a more unprofessio...
4    business owners stay away from this place an h...
Name: Review, dtype: object

In [9]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
RBC_Reviews['Review'] = RBC_Reviews['Review'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
RBC_Reviews['Review'].head()

0    found staff extremely helpful worked many fina...
1    extremely rude people work end phone needs ser...
2    ive rbc client nearly 20 years recently switch...
3    wow never dealt unprofessional business ever s...
4    business owners stay away place hour long inte...
Name: Review, dtype: object

In [10]:
freq = pd.Series(' '.join(RBC_Reviews['Review']).split()).value_counts()[:10]
freq

rbc        179
account    143
bank       108
credit      84
card        57
get         55
told        46
branch      45
time        43
money       42
dtype: int64

In [11]:
common_word=("rbc","account","bank","get","told","card")
RBC_Reviews['Review'] = RBC_Reviews['Review'].apply(lambda x: " ".join(x for x in x.split() if x not in common_word))
RBC_Reviews['Review'].head()


0    found staff extremely helpful worked many fina...
1    extremely rude people work end phone needs ser...
2    ive client nearly 20 years recently switched p...
3    wow never dealt unprofessional business ever s...
4    business owners stay away place hour long inte...
Name: Review, dtype: object

In [12]:
least = pd.Series(' '.join(RBC_Reviews['Review']).split()).value_counts()[-10:]
least

terrible      1
understand    1
b             1
mile          1
readily       1
workers       1
disability    1
laughable     1
anger         1
ruled         1
dtype: int64

In [13]:
least = list(least.index)
RBC_Reviews['Review'] = RBC_Reviews['Review'].apply(lambda x: " ".join(x for x in x.split() if x not in least))
RBC_Reviews['Review'].head()


0    found staff extremely helpful worked many fina...
1    extremely rude people work end phone needs ser...
2    ive client nearly 20 years recently switched p...
3    wow never dealt unprofessional business ever s...
4    business owners stay away place hour long inte...
Name: Review, dtype: object

In [14]:
RBC_Reviews['Review'] = RBC_Reviews['Review'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
RBC_Reviews['Review'].head()

0    found staff extremely helpful worked many fina...
1    extremely rude people work end phone need seri...
2    ive client nearly 20 year recently switched pe...
3    wow never dealt unprofessional business ever s...
4    business owner stay away place hour long inter...
Name: Review, dtype: object

In [15]:
RBC_Reviews['tokens'] = RBC_Reviews['Review'].apply(word_tokenize) 
RBC_Reviews

,Name,Rating,Date_Reviewed,Review,Province,tokens
0,"Tania of London, ON",4.0,"May 23, 2019",found staff extremely helpful worked many fina...,ON,"[found, staff, extremely, helpful, worked, man..."
1,"Terence of Thirnhill, ON",1.0,"May 23, 2019",extremely rude people work end phone need seri...,ON,"[extremely, rude, people, work, end, phone, ne..."
2,"John of Fort Erie, ON",2.0,"May 16, 2019",ive client nearly 20 year recently switched pe...,ON,"[ive, client, nearly, 20, year, recently, swit..."
3,"Keith of Calgary, AB",1.0,"May 10, 2019",wow never dealt unprofessional business ever s...,AB,"[wow, never, dealt, unprofessional, business, ..."
4,"Salman of Hamilton, ON",1.0,"May 5, 2019",business owner stay away place hour long inter...,ON,"[business, owner, stay, away, place, hour, lon..."
5,"marco of Hamilton, ON",1.0,"May 4, 2019",put deposit message went branch monday removed...,ON,"[put, deposit, message, went, branch, monday, ..."
6,"Sabah of Vancouver, BC",1.0,"April 30, 2019",business called request line credit wasting ho...,BC,"[business, called, request, line, credit, wast..."
7,"Pam of Brampton, ON",1.0,"April 30, 2019",franchising come everything neat package littl...,ON,"[franchising, come, everything, neat, package,..."
8,"KAREN of White Rock, BC",1.0,"April 29, 2019",read review would better understanding custome...,BC,"[read, review, would, better, understanding, c..."
9,"Jazz of Other, Other",1.0,"April 22, 2019",business deposit cheque every month cheque one...,Other,"[business, deposit, cheque, every, month, cheq..."


In [16]:
unlist_comments = [item for items in RBC_Reviews['tokens'] for item in items]
unlist_comments

['found',
 'staff',
 'extremely',
 'helpful',
 'worked',
 'many',
 'financial',
 'institution',
 'honestly',
 'agree',
 'customer',
 'likely',
 'provide',
 'bad',
 'review',
 'take',
 'time',
 'write',
 'good',
 'review',
 'warranted',
 'dealt',
 'royal',
 'past',
 'found',
 'staff',
 'offered',
 'product',
 'suited',
 'need',
 'made',
 'suggestion',
 'even',
 'didnt',
 'ask',
 'wealthy',
 'perfect',
 'credit',
 'extremely',
 'rude',
 'people',
 'work',
 'end',
 'phone',
 'need',
 'serious',
 'government',
 'oversight',
 'company',
 'many',
 'thing',
 'totally',
 'unethical',
 'charge',
 'fee',
 'service',
 'cancelled',
 'second',
 'call',
 'cancel',
 'fee',
 'clearly',
 'banking',
 'literally',
 'people',
 'dont',
 'look',
 'statement',
 'detail',
 'beware',
 'company',
 'care',
 'short',
 'term',
 'profit',
 'care',
 'employee',
 'customer',
 'find',
 'somewhere',
 'else',
 'business',
 'ive',
 'client',
 'nearly',
 '20',
 'year',
 'recently',
 'switched',
 'personal',
 'account',
 '

In [17]:
bigrams = nltk.collocations.BigramAssocMeasures()
bigramFinder = nltk.collocations.BigramCollocationFinder.from_words(unlist_comments)
bigram_freq = bigramFinder.ngram_fd.items()
bigramFreqTable = pd.DataFrame(list(bigram_freq), columns=['bigram','freq']).sort_values(by='freq', ascending=False)
bigramFreqTable

,bigram,freq
256,"(line, credit)",19
329,"(customer, service)",18
214,"(next, day)",10
176,"(2, week)",10
166,"(stay, away)",9
293,"(credit, card)",8
103,"(pay, bill)",8
202,"(went, branch)",7
619,"(2, year)",7
140,"(hour, phone)",6


In [18]:
en_stopwords = set(stopwords.words('english'))
#function to filter for ADJ/NN bigrams
def rightTypes(ngram):
    if '-pron-' in ngram or 't' in ngram:
        return False
    for word in ngram:
        if word in en_stopwords or word.isspace():
            return False
    acceptable_types = ('JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS')
    second_type = ('NN', 'NNS', 'NNP', 'NNPS')
    tags = nltk.pos_tag(ngram)
    if tags[0][1] in acceptable_types and tags[1][1] in second_type:
        return True
    else:
        return False
#filter bigrams
filtered_bi = bigramFreqTable[bigramFreqTable.bigram.map(lambda x: rightTypes(x))]
   
filtered_bi


,bigram,freq
256,"(line, credit)",19
329,"(customer, service)",18
214,"(next, day)",10
293,"(credit, card)",8
103,"(pay, bill)",8
140,"(hour, phone)",6
1710,"(customer, care)",6
435,"(poor, service)",6
183,"(credit, score)",6
679,"(unable, pay)",5


In [19]:
bi=filtered_bi.nlargest(20, 'freq')


In [20]:
bi.index = range(len(bi.index))
bi

,bigram,freq
0,"(line, credit)",19
1,"(customer, service)",18
2,"(next, day)",10
3,"(credit, card)",8
4,"(pay, bill)",8
5,"(hour, phone)",6
6,"(customer, care)",6
7,"(poor, service)",6
8,"(credit, score)",6
9,"(unable, pay)",5


In [ ]:
class Reviews(Base):
    __tablename__ = 'reviews'
    id = Column(Integer, primary_key=True)
    Name = Column(String)
    Rating = Column(String)
    Date_Reviewed=Column(String)
    Review=Column(String)
    Province=Column(String)

class Bigram(Base):
    __tablename__ = 'bigram'
    id = Column(Integer, primary_key=True)
    Bigram = Column(String)
    Freq = Column(Integer)



In [ ]:
engine = create_engine("sqlite:///customer_reviews.sqlite")
conn = engine.connect()

In [ ]:
Base.metadata.create_all(conn)

In [ ]:
session = Session(bind=engine)

In [ ]:
for index, item in RBC_Reviews.iterrows():
    review = Reviews(Name=item['Name'], Rating=item['Rating'],Date_Reviewed=item['Date_Reviewed'],Review=item['Review'],Province=item['Province'] )
    session.add(review)
    session.commit()



In [ ]:
pd.read_sql_query('select * from reviews', con=engine).head(10)

In [ ]:

for index,item in bi.iterrows():
    bigram = Bigram(Bigram=''.join(item['bigram']),Freq=int(item['freq']))
    session.add(bigram)
    session.commit()    
  

In [ ]:
pd.read_sql_query('select * from bigram', con=engine)